In [16]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import time 
import math
import sys
import os

In [31]:
def load_dataset():
    # Helper function for both Python 2 and python 3
    if sys.version_info[0] == 2:
        from urllib import urlretrieve
        import cpickle as pickle
        
        def pickle_load(f, encoding):
            return pickle.load(f)
        
    else:
        from urllib.request import urlretrieve
        import pickle
        
        def pickle_load(f, encoding):
            return pickle.load(f, encoding=encoding)
    
    # We will now downoload the data set if it is not yet available
    
    url = 'http://deeplearning.net/data/mnist/mnist.pkl.gz'
    filename = 'mnist.pkl.gz'
    if not os.path.exists(filename):
        print("Downloading MNIST Dataset ....")
        urlretrieve(url, filename)
        
    # We will now load and unpickle the file
    import gzip
    
    with gzip.open(filename, 'rb') as f:
        data = pickle_load(f, encoding='latin-1')
        
    # MNIST datset consists of 6 Numpy array
    # Input and targets for the training set, validation set and test set.
    
    X_train, y_train = data[0]
    X_val, y_val = data[1]
    X_test, y_test = data[2]
    """
    print(X_train.shape)
    print(X_val.shape)
    print(X_test.shape)
    """
    # The input come as vectors, we reshape them to monochrome 2D images.
    X_train = X_train.reshape((-1, 1, 28, 28))
    X_val = X_val.reshape((-1, 1, 28, 28))
    X_test = X_test.reshape((-1, 1, 28, 28))
    """
    print(X_train.shape)
    print(X_val.shape)
    print(X_test.shape)
    """
    # The targets sre int64, we cast them into int8 as GPU compatibility
    y_train = y_train.astype(np.uint8) 
    y_val = y_val.astype(np.uint8)
    y_test = y_test.astype(np.uint8)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [32]:
def preprocess_image(X, Y):
    pass

In [36]:
def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches

In [37]:
learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Load MNIST data
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

# The features are already scaled and the data is shuffled
train_features = X_train
test_features = X_test
val_features = X_val

train_labels = y_train
test_labels = y_test
val_labels = y_val

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
	
# TODO: Set batch size
batch_size = 64
assert batch_size is not None, 'You must set the batch size'

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)

    # TODO: Train optimizer on all batches
    # for batch_features, batch_labels in ______
    for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
        sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(accuracy, feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))


(50000, 784)
(10000, 784)
(10000, 784)
(50000, 1, 28, 28)
(10000, 1, 28, 28)
(10000, 1, 28, 28)
Instructions for updating:
Use `tf.global_variables_initializer` instead.


ValueError: Cannot feed value of shape (64, 1, 28, 28) for Tensor 'Placeholder_2:0', which has shape '(?, 784)'